# Code 1 - SQL Agent 
- Open API
- Langchain
- sqlite database

### Referances
- sqlite (download portable package) https://sqlitebrowser.org/dl/

## Step 1 - Connect to Database

### Import Packages

In [1]:
from sqlite3 import connect
from langchain_community.utilities.sql_database import SQLDatabase

### Create Connection URL

In [2]:
db = SQLDatabase.from_uri("sqlite:///chinook.db", sample_rows_in_table_info = 3)

### Check Databases 
- if the database is blank, please use sqlite browser to populate the same

In [3]:
print(len(db.get_usable_table_names()), db.get_usable_table_names())

11 ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [4]:
print(db.table_info)


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Empl

### Connect to sqlite database

In [5]:
con = connect("chinook.db")

### Check Sample Query

In [6]:
cur = con.execute("Select Count(Distinct(AlbumId)) from Album;")

In [7]:
print(cur.fetchall())

[(347,)]


## Step 2 - Setup Langchain SQL Chain

### Load Environment Variables 
- Mostly API Keys

In [8]:
#import os
#from dotenv import load_dotenv
#load_dotenv()

In [9]:
#API_KEY = os.getenv("OPENAI_API_KEY")

### Load Packages

In [11]:
from langchain_ollama.llms import OllamaLLM

### Create an Instance of LLM with configuration

In [12]:
llm = OllamaLLM(model="llama3.1")  # mixtral

### Check LLM 

In [13]:
llm

OllamaLLM(model='llama3.1')

## Step 3 - Create Chain

### Import Packages

In [14]:
from langchain_experimental.sql.base import SQLDatabaseChain

### Setup SQL DB Chain

In [15]:
sql_db_chain = SQLDatabaseChain.from_llm(llm = llm, db = db, verbose = True)

### Run and Print Chain Output

In [16]:
result = sql_db_chain.invoke("What is the total count of customers?")
print(result)



> Entering new SQLDatabaseChain chain...
What is the total count of customers?
SQLQuery:SELECT COUNT("CustomerId") FROM "Invoice"
SQLResult: [(412,)]
Answer:Based on the provided data, we can see that there are 3 rows in the Invoice table with CustomerId values of 2, 4, and 8. To find the total count of customers, we need to consider all unique CustomerId values across all tables.

Let's analyze the data:

* In the Invoice table, there are 3 rows with CustomerId values of 2, 4, and 8.
* There might be other tables that also have CustomerId values. However, upon reviewing the provided schema, I see no such references.

Considering this information, we can conclude that there are at least 3 unique customers (with IDs 2, 4, and 8). To find the total count of customers, we need to consider all possible tables with CustomerId columns. Based on the given schema, I did not find any other tables with a reference to CustomerId.

Given this context, it's reasonable to assume that the provided 

### Check Template

In [16]:
print(sql_db_chain.llm_chain.prompt.template)

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: 

# END - Next Topic - SQL Agents